In [1]:
import pandas as pd
from tabulate import tabulate
from wmfdata import hive, mariadb

You can find the source for `wmfdata` at https://github.com/neilpquinn/wmfdata


In [2]:
# First month (in "YYYY-MM-01" format) for which we want data
START = "2017-07-01"
# When to truncate data, to avoid a final partial month
END = "2018-11-01"

In [3]:
edits = mariadb.run("""
select
    month,
    sum(mobile_web_edits + mobile_app_edits) as wikipedia_mobile_edits,
    sum(if(wiki = "enwiki", mobile_web_edits + mobile_app_edits, 0)) as enwiki_mobile_edits,
    sum(mobile_app_edits) as wikipedia_app_edits
from staging.editor_month
inner join enwiki.sites
on site_global_key = wiki
where
    month >= "{start}" and
    site_group = "wikipedia"
group by month
""".format(start = START))

In [4]:
edits = edits.assign(month=lambda df: pd.to_datetime(df["month"]))
edits.tail()

,month,wikipedia_mobile_edits,enwiki_mobile_edits,wikipedia_app_edits
11,2018-06-01,1059306.0,431778.0,78504.0
12,2018-07-01,1106156.0,452541.0,87498.0
13,2018-08-01,1160279.0,463675.0,94059.0
14,2018-09-01,1078723.0,427122.0,84893.0
15,2018-10-01,1159329.0,453191.0,92435.0


In [ ]:
wp_article_edits_sql = """
select
    database() as wiki,
    left(coalesce(rev_timestamp, ar_timestamp), 6) as month,
    sum(coalesce(page_namespace, ar_namespace) = 0) as main_ns_edits
from tag_summary
left join revision
on ts_rev_id = rev_id
left join archive 
on ar_rev_id = rev_id
left join page
on rev_page = page_id
where 
    ts_tags like "%mobile%" and 
    coalesce(rev_timestamp, ar_timestamp) between "{start}" and "{end}"
group by left(coalesce(rev_timestamp, ar_timestamp), 6);
""".format(
    start = START.replace("-", ""),
    end = END.replace("-", "")
)

wps = mariadb.list_wikis("wikipedia")

wp_article_edits = mariadb.multirun(wp_article_edits_sql, wps)

aawiki completed in 0 s
abwiki completed in 1 s
acewiki completed in 1 s
adywiki completed in 0 s
afwiki completed in 2 s
akwiki completed in 0 s
alswiki completed in 1 s
amwiki completed in 0 s
angwiki completed in 0 s
anwiki completed in 1 s
arcwiki completed in 1 s
arwiki completed in 394 s
arzwiki completed in 2 s
astwiki completed in 5 s
aswiki completed in 1 s
atjwiki completed in 0 s
avwiki completed in 1 s
aywiki completed in 0 s
azbwiki completed in 6 s
azwiki completed in 9 s
barwiki completed in 1 s
bat_smgwiki completed in 0 s
bawiki completed in 2 s
bclwiki completed in 0 s
be_x_oldwiki completed in 5 s
bewiki completed in 4 s
bgwiki completed in 15 s
bhwiki completed in 3 s
biwiki completed in 1 s
bjnwiki completed in 0 s
bmwiki completed in 0 s
bnwiki completed in 14 s
bowiki completed in 0 s
bpywiki completed in 1 s
brwiki completed in 2 s
bswiki completed in 4 s
bugwiki completed in 0 s
bxrwiki completed in 0 s
cawiki completed in 98 s
cbk_zamwiki completed in 0 s
cdow

In [9]:
wp_article_edits.head()

,wiki,month,main_ns_edits
0,abwiki,201707,0.0
1,abwiki,201709,0.0
2,abwiki,201710,1.0
3,abwiki,201711,0.0
4,abwiki,201712,4.0


In [10]:
mobile_edits = (
    wp_article_edits
    .groupby("month").aggregate({"main_ns_edits": "sum"})
    .reset_index()
    .assign(month=lambda df: pd.to_datetime(df["month"], format="%Y%m"))
    .merge(edits, on = "month")
    .assign(
        non_article_edits=lambda df: df["wikipedia_mobile_edits"] - df["main_ns_edits"],
        non_enwiki_edits=lambda df: df["wikipedia_mobile_edits"] - df["enwiki_mobile_edits"]
    )
    .reindex(columns=[
        "month", "wikipedia_mobile_edits", 
        "main_ns_edits", "non_article_edits", 
        "enwiki_mobile_edits", "non_enwiki_edits",
        "wikipedia_app_edits"
    ])
    .set_index("month")
    .to_period()
)

mobile_edits

,wikipedia_mobile_edits,main_ns_edits,non_article_edits,enwiki_mobile_edits,non_enwiki_edits,wikipedia_app_edits
month,,,,,,
2017-07,886826.0,804698.0,82128.0,382019.0,504807.0,83384.0
2017-08,952378.0,856678.0,95700.0,399603.0,552775.0,84192.0
2017-09,863807.0,776230.0,87577.0,366213.0,497594.0,76036.0
2017-10,938436.0,847105.0,91331.0,401901.0,536535.0,80361.0
2017-11,928294.0,833859.0,94435.0,396453.0,531841.0,75150.0
2017-12,963289.0,862530.0,100759.0,402423.0,560866.0,77713.0
2018-01,1088852.0,978289.0,110563.0,448648.0,640204.0,80595.0
2018-02,954162.0,848690.0,105472.0,381474.0,572688.0,65336.0
2018-03,1055729.0,938661.0,117068.0,437521.0,618208.0,75202.0


In [11]:
print(tabulate(mobile_edits, headers="keys", tablefmt="pipe", floatfmt=".0f"))

| month   |   wikipedia_mobile_edits |   main_ns_edits |   non_article_edits |   enwiki_mobile_edits |   non_enwiki_edits |   wikipedia_app_edits |
|:--------|-------------------------:|----------------:|--------------------:|----------------------:|-------------------:|----------------------:|
| 2017-07 |                   886826 |          804698 |               82128 |                382019 |             504807 |                 83384 |
| 2017-08 |                   952378 |          856678 |               95700 |                399603 |             552775 |                 84192 |
| 2017-09 |                   863807 |          776230 |               87577 |                366213 |             497594 |                 76036 |
| 2017-10 |                   938436 |          847105 |               91331 |                401901 |             536535 |                 80361 |
| 2017-11 |                   928294 |          833859 |               94435 |                396453 |          